In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import pyspark

try: 
    type(sc)
except NameError:
    sc = pyspark.SparkContext('local[*]')

In [3]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

import pyspark_csv as pycsv
sc.addPyFile('pyspark_csv.py')
plaintext_rdd = sc.textFile('propiedades-clean.csv')
dataframe = pycsv.csvToDataFrame(sqlContext, plaintext_rdd)
data = dataframe.rdd

In [4]:
import re, string, unicodedata
def removePunctuation(text):
    
    text=str(text).lower().strip(' ')
    text=unicodedata.normalize('NFD', text).encode('ascii','ignore')
    text=text.decode('utf-8')
    text=re.sub(r'[^a-zA-Z0-9 ]', '', text)   
    return text


In [5]:
i = 0
for row in dataframe:
    print(i, "--->", row)
    i += 1

0 ---> Column<b''>
1 ---> Column<b'created_on'>
2 ---> Column<b'year'>
3 ---> Column<b'currency'>
4 ---> Column<b'description'>
5 ---> Column<b'expenses'>
6 ---> Column<b'floor'>
7 ---> Column<b'geonames_id'>
8 ---> Column<b'id'>
9 ---> Column<b'lat'>
10 ---> Column<b'lon'>
11 ---> Column<b'operation'>
12 ---> Column<b'place_name'>
13 ---> Column<b'place_with_parent_names'>
14 ---> Column<b'price'>
15 ---> Column<b'price_aprox_local_currency'>
16 ---> Column<b'price_aprox_usd'>
17 ---> Column<b'price_per_m2'>
18 ---> Column<b'price_usd_per_m2'>
19 ---> Column<b'property_type'>
20 ---> Column<b'rooms'>
21 ---> Column<b'state_name'>
22 ---> Column<b'surface_covered_in_m2'>
23 ---> Column<b'surface_total_in_m2'>


In [6]:
# id, rooms, description
asd = data.map(lambda x: (x[8], x[20], x[4]))
def formato(x):
    if x[1] == None:
        return((x[0],0,x[2]))
    return x
asd = asd.map(formato)
asd.count()

221341

In [7]:
idYdes = asd.map(lambda x: (x[0], x[1], removePunctuation(x[2])))
antes = idYdes.count()
idYdes = idYdes.filter(lambda x: x[2] != None)
final = antes - idYdes.count()
print("Datos con descripcion nula: " + str(final))

Datos con descripcion nula: 0


In [8]:
def letrasANumero(word):
    dic = {
        'mono': '1',
        'uno': '1',
        'un': '1',
        'dos': '2',
        'tres': '3',
        'cuatro': '4',
        'cinco': '5',
        'seis': '6',
        'siete': '7',
        'ocho': '8'
    }
    if word in dic:
        return dic[word]
    else:
        return word
    

def buscarBigramas(s, *words):
    r = 0
    l = s.split()
    for i in range(0,len(l)-1):
        try:
            n = int(letrasANumero(l[i]))
            if l[i+1] in words:
                if n > r:
                    r = n
        except ValueError:
            None
        try:
            n = int(letrasANumero(l[i+1]))
            if l[i] == words:
                if int(n) > r:
                    r = n
        except ValueError:
            None
    return r

In [9]:
idYdesFilter = idYdes.map(lambda x: (x[0], x[1], buscarBigramas(x[2],'ambientes','ambiente')))
idYdesFilter = idYdesFilter.filter(lambda x: x[1] < x[2])
idYdesFilter = idYdesFilter.filter(lambda x: x[2] < 15)
idYdesFilter.count()

39969

In [18]:
df = idYdesFilter.collect()
import pandas as pd
df = pd.DataFrame(df)

In [11]:
df.columns = ['id', 'Ambientes', 'Abientes Actualizado']

In [16]:
df.head(10)
df = df.drop(df[['Ambientes']], axis=1)
df.columns = ['id', 'rooms']

In [17]:
df.to_csv('ambientesActualizados.csv')